In [0]:
#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
#!pip3 install torchvision

In [0]:
import time

import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from sklearn.metrics import accuracy_score

import numpy as np
import PIL
import random
from IPython.core.display import Image, display

import matplotlib.pyplot as plt

# For showing good progress bars.
import tqdm

In [0]:
#отключает warnings pytorch
import warnings
warnings.filterwarnings('ignore')

In [4]:
print(torch.__version__)
print(torch.cuda.is_available())

print(torch.cuda.device_count(), torch.cuda.get_device_name(0))

print(torch.cuda.current_device())
print(torch.cuda.device(0))
print("CudaVersion : ",torch.version.cuda)

1.0.1.post2
True
1 Tesla K80
0
CudaVersion :  10.0.130


In [0]:
DEVICE_ID = 0
DEVICE = torch.device('cuda:%d' % DEVICE_ID)
torch.cuda.set_device(DEVICE_ID)

### Для запуска без GPU раскомментировать и закоментировать код выше
# DEVICE = torch.device('cpu')

In [0]:
np.random.seed(100500)

def data2image(data):
    res = np.transpose(np.reshape(data ,(3, 32,32)), (1,2,0))
    return PIL.Image.fromarray(np.uint8(res))

def imshow(img):
    if isinstance(img, torch.Tensor): img = img.numpy().astype('uint8')
    plt.imshow(np.transpose(img, (1, 2, 0)))
    
def prediction2classes(output_var):
    _, predicted = torch.max(output_var.data, 1)
    predicted.squeeze_()
    classes = predicted.tolist()
    return classes

def make_solution_pytorch(net, input_tensor, a_batch_size):
    res = []
    net = net.eval()
    cur_pos = 0
    while cur_pos <= len(input_tensor):
        outputs = net(input_tensor[cur_pos:cur_pos+a_batch_size])
        res += prediction2classes(outputs)
        cur_pos += a_batch_size
    return res

In [0]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

class CifarDataset(Dataset):
    def __init__(self, input_path, is_train=True, transform=None):
                        
        data = np.load(input_path)
        if is_train: 
            self.Y, self.X = np.hsplit(data, [1]) 
            self.Y = [item[0] for item in self.Y]
        else: 
            self.X = data
            self.Y = None
            
        self.X = self.X.reshape((self.X.shape[0], 3, 32, 32))
        self.X = self.X.transpose((0, 2, 3, 1)) #приводим к виду (N, H, W, C)
        self.X = [Image.fromarray(img) for img in self.X]
                
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        
        sample = self.X[idx]

        if self.transform: sample = self.transform(sample)

        if self.Y is None: return sample
        else: return (sample, self.Y[idx])

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
#тут папки с проблема должны быть заключены в ""
PATH_OF_DATA= '/content/gdrive/"My Drive"/"Technotrack_hw_4"/'
!ls {PATH_OF_DATA}

 homework_4_no_classes.test.npy     my_solution_resnet_wide_6.csv
 homework_4.train.npy		    my_solution_wide_r_1.csv
 Kaggle_HW4.ipynb		    my_solution_wide_r_2.csv
'Kaggle_HW4_ResNet_py3 (1).ipynb'   my_solution_wide_r_3.csv
 my_damn_results		    my_solution_wide_r_5.csv
 my_solution_lenet.csv		    my_solution_wide_r_6.csv
 my_solution_res_2.csv		   'ResNet_HW4 (3).ipynb'
 my_solution_res_3.csv		    ResNet_HW4.ipynb
 my_solution_res.csv		    sample_submission.csv
 my_solution_resnet_wide_4.csv


In [0]:
#Тут папки с пробелами НЕ надо заключать в ""
DATA_PATH  = '/content/gdrive/My Drive/Technotrack_hw_4/'
train_path = 'homework_4.train.npy'
test_path  = 'homework_4_no_classes.test.npy'

In [0]:
np_mean = np.mean([item[0].numpy() for item in CifarDataset(DATA_PATH + train_path, transform=transforms.ToTensor())], axis=(0,2,3))
np_std = np.std([item[0].numpy() for item in CifarDataset(DATA_PATH + train_path, transform=transforms.ToTensor())], axis=(0,2,3))

In [12]:
np_mean, np_mean.shape

(array([0.50760865, 0.48708203, 0.44149536], dtype=float32), (3,))

In [13]:
np_std, np_std.shape

(array([0.26764476, 0.2567687 , 0.27647924], dtype=float32), (3,))

In [0]:
cifar_transform_norm = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(torch.FloatTensor(np_mean), torch.FloatTensor(np_std))
]
)

cifar_test_transform_norm = transforms.Compose([    
    transforms.ToTensor(),
    transforms.Normalize(torch.FloatTensor(np_mean), torch.FloatTensor(np_std))
]
)

In [0]:
dataset_train_norm = CifarDataset(DATA_PATH + train_path, transform=cifar_transform_norm)
dataloader_train_norm = DataLoader(dataset_train_norm, batch_size=128,
                        shuffle=True, num_workers=4)

dataset_test_norm = CifarDataset(DATA_PATH + test_path, is_train=False, transform=cifar_test_transform_norm)
dataloader_test_norm = DataLoader(dataset_test_norm, batch_size=128,
                        shuffle=False, num_workers=1)


def train_network(a_net, 
                  a_device,
                  dataloader_train_norm=dataloader_train_norm,
                  a_epochs=164,
                  a_batch_size=128,
                  a_lr=0.1):
    
    start_time = time.time()
    train_acc = []
    #a_net = torch.load('/content/gdrive/My Drive/my_damn_results_2_20x5')
    net = a_net.to(a_device)
    
    # Google Colab warks quite bizzare. It has a tendency to lose connection. From
    # here comes the main importance to store a model's parameters not to lose all results.
    #net.load_state_dict(torch.load('/content/gdrive/My Drive/my_damn_results_2_20x5'))
    #net = torch.load('/content/gdrive/My Drive/my_damn_results_2_20x5')

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(a_net.parameters(), lr=a_lr, weight_decay=0.0001, momentum=0.9)

    prev_epoch_time = start_time
    
    my_solution = 0
    
    for epoch in range(93, a_epochs):  # loop over the dataset multiple times
#         if epoch == 82:
#             optimizer = torch.optim.SGD(a_net.parameters(), lr=a_lr/10, weight_decay=0.0001, momentum=0.9) 
#         elif epoch == 123:
#             optimizer = torch.optim.SGD(a_net.parameters(), lr=a_lr/100, weight_decay=0.0001, momentum=0.9)

        if epoch == 62:
            optimizer = torch.optim.SGD(a_net.parameters(), lr=a_lr/10, weight_decay=0.0001, momentum=0.9) 
        elif epoch == 93:
            optimizer = torch.optim.SGD(a_net.parameters(), lr=a_lr/100, weight_decay=0.0001, momentum=0.9) 
        elif epoch == 133:
            optimizer = torch.optim.SGD(a_net.parameters(), lr=a_lr/1000, weight_decay=0.0001, momentum=0.9) 
        
        net = net.train()        
        epoch_accuracy = 0.0
        epoch_iters = 0
        for item in tqdm.tqdm(dataloader_train_norm):
          
            epoch_iters += 1
            
            inputs = item[0].to(a_device)
            labels = item[1].long().to(a_device)
            #print('Sizes:', inputs.size(), labels.size())
            #print(inputs)
            #print(labels)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            #print(outputs.size())
            #print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
#             prediction2classes(outputs)

#             print(labels.cpu())
#             print(prediction2classes(outputs))
            epoch_accuracy += accuracy_score(labels.cpu(), prediction2classes(outputs))
            #print(accuracy_score(labels.cpu(), prediction2classes(outputs)))

        epoch_accuracy /= epoch_iters
        train_acc.append(epoch_accuracy)
        
        # Make a solution after each epoch only to swiftly pass it to kaggle.
        my_solution = make_solution(resnet, DEVICE)
        
        print("Epoch ", epoch, round(train_acc[-1], 4))
        cur_epoch_time = time.time()
        print('Epoch time : ', cur_epoch_time - prev_epoch_time )
        prev_epoch_time = cur_epoch_time
        
        # Save modal's params after each epoch.
        torch.save(net.state_dict(), '/content/gdrive/My Drive/my_damn_results_3_20x5')

    print('Finished Training')
    print("Total time : ", (time.time()-start_time))
    
    plt.plot(train_acc, label='Train')
    plt.legend()
#     plt.grid()
    plt.grid(c='grey')

In [0]:
def make_solution(a_net, a_device):
    res = []
    net = a_net.eval()
    for item in tqdm.tqdm(dataloader_test_norm):
        inputs = item.to(a_device)
        outputs = net(inputs) 

        res += prediction2classes(outputs)
    return res

The usual ResNet showed not appropriate results on the dataset given. That is why after a short research in the Internet the solution with wide ResNet was found. I tried this very implementation and it worked quite good, yeah!


In [0]:
class wide_ResBlock(nn.Module):
    def __init__(self, a_in_planes, planes, dropout_rate, stride=1):
        super(wide_ResBlock, self).__init__()
        
        self.BN1 = nn.BatchNorm2d(a_in_planes)
        self.Conv1 = nn.Conv2d(a_in_planes, planes, kernel_size=3, padding=1, bias=True)
        self.Dropout = nn.Dropout(p=dropout_rate)
        self.BN2 = nn.BatchNorm2d(planes)
        self.Conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.shortcut = nn.Sequential()
        if (stride != 1) or (a_in_planes != planes):
            self.shortcut = nn.Sequential(
                nn.Conv2d(a_in_planes, planes, kernel_size=1, stride=stride, bias=True),
            )

    def forward(self, x):
        out = self.BN1(x)
        out = F.relu(out)
        out = self.Conv1(out)
        out = self.Dropout(out)
        out = self.BN2(out)
        out = F.relu(out)
        out = self.Conv2(out)
        out += self.shortcut(x)
        return out

In [0]:
class Wide_ResNet(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(Wide_ResNet, self).__init__()
        self.in_planes = 16
        n = (depth - 4) / 6
        self.Conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=True)
        self.WideLayer1 = self._wide_layer(wide_ResBlock, 16 * widen_factor, n, dropout_rate, stride=1)
        self.WideLayer2 = self._wide_layer(wide_ResBlock, 32 * widen_factor, n, dropout_rate, stride=2)
        self.WideLayer3 = self._wide_layer(wide_ResBlock, 64 * widen_factor, n, dropout_rate, stride=2)
        self.BN1 = nn.BatchNorm2d(64 * widen_factor, momentum=0.9)
        self.Dense = nn.Linear(64 * widen_factor, num_classes)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1] * int(num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.Conv1(x)
        out = self.WideLayer1(out)
        out = self.WideLayer2(out)
        out = self.WideLayer3(out)
        out = self.BN1(out)
        out = F.relu(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.Dense(out)
        return out

In [0]:
resnet = Wide_ResNet(28, 5, 0.3, 100)
train_network(resnet, torch.device(DEVICE))

 48%|████▊     | 189/391 [03:52<04:06,  1.22s/it]

# Важно переключить сеть в режим eval - иначе dropout будет работать некорректно

In [0]:
# def make_solution(a_net, a_device):
#     res = []
#     net = a_net.eval()
#     for item in tqdm.tqdm(dataloader_test_norm):
#         inputs = item.to(a_device)
#         outputs = net(inputs) 

#         res += prediction2classes(outputs)
#     return res

In [0]:
# my_solution = make_solution(dense_net, DEVICE)
my_solution = make_solution(resnet, DEVICE)

In [0]:
file_name = DATA_PATH + 'my_solution_wide_r_6.csv'

with open(file_name, 'w') as fout:
    print('Id', 'Prediction', sep=',', file=fout)
    for i, prediction in enumerate(my_solution):
        print(i, prediction, sep=',', file=fout)
        
# from google.colab import files
# files.download(file_name)